## Import Required Libraries

In [3]:
import cv2
import numpy as np
import face_recognition
import math
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image   # for preprocessing the images

from tensorflow.keras.utils import to_categorical 
from skimage.transform import resize   # for resizing images

import timeit
import time
import tqdm
from tensorflow.keras.preprocessing import image
from PIL import Image
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os

In [4]:
videoFile = "/Users/klrao/Downloads/1-Dissertation-DeepFake/train_sample_videos/etmcruaihe.mp4"
outpath = "/Users/klrao/Downloads/1-Dissertation-DeepFake/train_sample_videos/OutputFrames/"

##  Read Metadata JSON

In [11]:
os.chdir('/Users/klrao/Downloads/1-Dissertation-DeepFake/train_sample_videos')
metadata_df = pd.read_json('metadata.json')
metadata_df = metadata_df.T
print(metadata_df)

               label  split        original
aagfhgtpmv.mp4  FAKE  train  vudstovrck.mp4
aapnvogymq.mp4  FAKE  train  jdubbvfswz.mp4
abarnvbtwb.mp4  REAL  train            None
abofeumbvv.mp4  FAKE  train  atvmxvwyns.mp4
abqwwspghj.mp4  FAKE  train  qzimuostzz.mp4
...              ...    ...             ...
etejaapnxh.mp4  FAKE  train  wtreibcmgm.mp4
etmcruaihe.mp4  FAKE  train  afoovlsmtx.mp4
etohcvnzbj.mp4  FAKE  train  bdnaqemxmr.mp4
eudeqjhdfd.mp4  REAL  train            None
eukvucdetx.mp4  FAKE  train  gjypopglvi.mp4

[400 rows x 3 columns]


### Select Only Few Fakes

In [78]:
metadata_df.loc[(metadata_df.index=="etmcruaihe.mp4") | (metadata_df.index=="afoovlsmtx.mp4")
               |(metadata_df.index=="acqfdwsrhi.mp4") | (metadata_df.index=="ccfoszqabv.mp4")
               |(metadata_df.index=="cxrfacemmq.mp4") | (metadata_df.index=="dlpoieqvfb.mp4")
               |(metadata_df.index=="bvzjkezkms.mp4") | (metadata_df.index=="ekcrtigpab.mp4") 
               ]



,label,split,original
acqfdwsrhi.mp4,FAKE,train,ccfoszqabv.mp4
afoovlsmtx.mp4,REAL,train,None
bvzjkezkms.mp4,FAKE,train,ekcrtigpab.mp4
ccfoszqabv.mp4,REAL,train,None
cxrfacemmq.mp4,FAKE,train,dlpoieqvfb.mp4
dlpoieqvfb.mp4,REAL,train,None
ekcrtigpab.mp4,REAL,train,None
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4


## Capture Videos

In [80]:
selected_train_df = metadata_df.loc[(metadata_df.index=="etmcruaihe.mp4") | (metadata_df.index=="afoovlsmtx.mp4")
               |(metadata_df.index=="acqfdwsrhi.mp4") | (metadata_df.index=="ccfoszqabv.mp4")
               |(metadata_df.index=="cxrfacemmq.mp4") | (metadata_df.index=="dlpoieqvfb.mp4")
               |(metadata_df.index=="bvzjkezkms.mp4") | (metadata_df.index=="ekcrtigpab.mp4") 
               ]
selected_train_df.shape

(8, 3)

In [81]:
selected_train_df

,label,split,original
acqfdwsrhi.mp4,FAKE,train,ccfoszqabv.mp4
afoovlsmtx.mp4,REAL,train,None
bvzjkezkms.mp4,FAKE,train,ekcrtigpab.mp4
ccfoszqabv.mp4,REAL,train,None
cxrfacemmq.mp4,FAKE,train,dlpoieqvfb.mp4
dlpoieqvfb.mp4,REAL,train,None
ekcrtigpab.mp4,REAL,train,None
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4


In [82]:
selected_train_df.index[0],selected_train_df.index[1],selected_train_df.index[2]

('acqfdwsrhi.mp4', 'afoovlsmtx.mp4', 'bvzjkezkms.mp4')

In [85]:
# storing the frames from training videos
for i in tqdm(range(selected_train_df.shape[0])):
    count = 0
    videoFile = selected_train_df.index[i]
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frameRate = cap.get(5) #frame rate
    #print("frameRate",frameRate)
    x=1
    c = 0
    #def generator():
    frame_number = 0  
    #    yield
    start = time.time()  
    #print(start)
    frames = []
    r_frames = []
    train_image = []
    faces = []
    while((cap.isOpened())):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        r_frames.append(frame)
        frame_number+=1
        #print("FId",frameId)
        #print("frame_number",frame_number)
        if(frame_number+1 < 300):
            height, width, layers = frame.shape
            size = (width,height)
            #print('Shape',frame.shape)
            face_locations = face_recognition.face_locations(frame)
            #print("face_locations",face_locations)
            #t ,r ,b, l = face_locations
            for top, right, bottom, left in face_locations:
                #print("trbl",top, right, bottom, left)
                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, "name", (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

                face_image = frame[top:bottom, left:right]
                #print('Writing faceimage',face_image.shape)

            face_landmarks_list = face_recognition.face_landmarks(frame)
            for face_landmark in face_landmarks_list:
                left_eye = face_landmark['left_eye']
                right_eye = face_landmark['right_eye']

                color = (255,0,0)
                thickness = 2

                cv2.rectangle(frame, left_eye[0], right_eye[-1], color, thickness)

        if (ret != True):
            print(frameId,ret)
            break
        '''if (frameId % math.floor(frameRate) == 0):
            print(c)
            filename = outpath + "frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
        '''
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            frame_file = outpath + videoFile +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(frame_file, frame)
        print("Writing frame {} / {}".format(frame_number, length))

        frames.append(frame)
        faces.append(face_image)

    cap.release()
    print ("Done!")
    end = time.time()
    #print(end)
    #print("Time",end - start)

  0%|          | 0/8 [00:00<?, ?it/s]

Writing frame 1 / 300
Writing frame 2 / 300
Writing frame 3 / 300
Writing frame 4 / 300
Writing frame 5 / 300
Writing frame 6 / 300
Writing frame 7 / 300
Writing frame 8 / 300
Writing frame 9 / 300
Writing frame 10 / 300
Writing frame 11 / 300
Writing frame 12 / 300
Writing frame 13 / 300
Writing frame 14 / 300
Writing frame 15 / 300
Writing frame 16 / 300
Writing frame 17 / 300
Writing frame 18 / 300
Writing frame 19 / 300
Writing frame 20 / 300
Writing frame 21 / 300
Writing frame 22 / 300
Writing frame 23 / 300
Writing frame 24 / 300
Writing frame 25 / 300
Writing frame 26 / 300
Writing frame 27 / 300
Writing frame 28 / 300
Writing frame 29 / 300
Writing frame 30 / 300
Writing frame 31 / 300
Writing frame 32 / 300
Writing frame 33 / 300
Writing frame 34 / 300
Writing frame 35 / 300
Writing frame 36 / 300
Writing frame 37 / 300
Writing frame 38 / 300
Writing frame 39 / 300
Writing frame 40 / 300
Writing frame 41 / 300
Writing frame 42 / 300
Writing frame 43 / 300
Writing frame 44 / 3

 12%|█▎        | 1/8 [11:42<1:21:57, 702.54s/it]

Writing frame 298 / 300
Writing frame 299 / 300
Writing frame 300 / 300
300.0 False
Done!
Writing frame 1 / 300
Writing frame 2 / 300
Writing frame 3 / 300
Writing frame 4 / 300
Writing frame 5 / 300
Writing frame 6 / 300
Writing frame 7 / 300
Writing frame 8 / 300
Writing frame 9 / 300
Writing frame 10 / 300
Writing frame 11 / 300
Writing frame 12 / 300
Writing frame 13 / 300
Writing frame 14 / 300
Writing frame 15 / 300
Writing frame 16 / 300
Writing frame 17 / 300
Writing frame 18 / 300
Writing frame 19 / 300
Writing frame 20 / 300
Writing frame 21 / 300
Writing frame 22 / 300
Writing frame 23 / 300
Writing frame 24 / 300
Writing frame 25 / 300
Writing frame 26 / 300
Writing frame 27 / 300
Writing frame 28 / 300
Writing frame 29 / 300
Writing frame 30 / 300
Writing frame 31 / 300
Writing frame 32 / 300
Writing frame 33 / 300
Writing frame 34 / 300
Writing frame 35 / 300
Writing frame 36 / 300
Writing frame 37 / 300
Writing frame 38 / 300
Writing frame 39 / 300
Writing frame 40 / 300

 25%|██▌       | 2/8 [23:29<1:10:22, 703.80s/it]

...

Writing frame 298 / 300
Writing frame 299 / 300
Writing frame 300 / 300
300.0 False
Done!
Writing frame 1 / 300
Writing frame 2 / 300
Writing frame 3 / 300
Writing frame 4 / 300
Writing frame 5 / 300
Writing frame 6 / 300
Writing frame 7 / 300
Writing frame 8 / 300
Writing frame 9 / 300
Writing frame 10 / 300
Writing frame 11 / 300
Writing frame 12 / 300
Writing frame 13 / 300
Writing frame 14 / 300
Writing frame 15 / 300
Writing frame 16 / 300
Writing frame 17 / 300
Writing frame 18 / 300
Writing frame 19 / 300
Writing frame 20 / 300
Writing frame 21 / 300
Writing frame 22 / 300
Writing frame 23 / 300
Writing frame 24 / 300
Writing frame 25 / 300
Writing frame 26 / 300
Writing frame 27 / 300
Writing frame 28 / 300
Writing frame 29 / 300
Writing frame 30 / 300
Writing frame 31 / 300
Writing frame 32 / 300
Writing frame 33 / 300
Writing frame 34 / 300
Writing frame 35 / 300
Writing frame 36 / 300
Writing frame 37 / 300
Writing frame 38 / 300
Writing frame 39 / 300
Writing frame 40 / 300

 88%|████████▊ | 7/8 [1:25:06<12:20, 740.88s/it]

Writing frame 298 / 300
Writing frame 299 / 300
Writing frame 300 / 300
300.0 False
Done!
Writing frame 1 / 300
Writing frame 2 / 300
Writing frame 3 / 300
Writing frame 4 / 300
Writing frame 5 / 300
Writing frame 6 / 300
Writing frame 7 / 300
Writing frame 8 / 300
Writing frame 9 / 300
Writing frame 10 / 300
Writing frame 11 / 300
Writing frame 12 / 300
Writing frame 13 / 300
Writing frame 14 / 300
Writing frame 15 / 300
Writing frame 16 / 300
Writing frame 17 / 300
Writing frame 18 / 300
Writing frame 19 / 300
Writing frame 20 / 300
Writing frame 21 / 300
Writing frame 22 / 300
Writing frame 23 / 300
Writing frame 24 / 300
Writing frame 25 / 300
Writing frame 26 / 300
Writing frame 27 / 300
Writing frame 28 / 300
Writing frame 29 / 300
Writing frame 30 / 300
Writing frame 31 / 300
Writing frame 32 / 300
Writing frame 33 / 300
Writing frame 34 / 300
Writing frame 35 / 300
Writing frame 36 / 300
Writing frame 37 / 300
Writing frame 38 / 300
Writing frame 39 / 300
Writing frame 40 / 300

100%|██████████| 8/8 [1:37:16<00:00, 729.56s/it]

Writing frame 298 / 300
Writing frame 299 / 300
Writing frame 300 / 300
300.0 False
Done!


In [87]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from tensorflow.keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [92]:
os.chdir("/Users/klrao/Downloads/1-Dissertation-DeepFake/train_sample_videos/OutputFrames/") 

In [112]:
# getting the names of all the images
images = glob("*.jpg")
train_image = []
train_class = []
print(len(images))
for i in tqdm(range(len(images))):
    print(images[i])
    # creating the image name
    train_image.append(images[i])
    #print(images[i])
    # creating the class of image
    
    a = selected_train_df[selected_train_df.index == images[i].split('_')[0]]['label']
    print(a)
    train_class.append(a.values[0])
    #print(images[i].split('_')[0]+"->"+selected_train_df[selected_train_df.index == images[i].split('_')[0]]['label'])
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('/Users/klrao/Downloads/1-Dissertation-DeepFake/train_sample_videos/OutputFrames/train_new.csv',header=True, index=False)

100%|██████████| 88/88 [00:00<00:00, 974.72it/s]

88
ccfoszqabv.mp4_frame2.jpg
ccfoszqabv.mp4    REAL
Name: label, dtype: object
bvzjkezkms.mp4_frame5.jpg
bvzjkezkms.mp4    FAKE
Name: label, dtype: object
bvzjkezkms.mp4_frame4.jpg
bvzjkezkms.mp4    FAKE
Name: label, dtype: object
ccfoszqabv.mp4_frame3.jpg
ccfoszqabv.mp4    REAL
Name: label, dtype: object
ccfoszqabv.mp4_frame1.jpg
ccfoszqabv.mp4    REAL
Name: label, dtype: object
ekcrtigpab.mp4_frame9.jpg
ekcrtigpab.mp4    REAL
Name: label, dtype: object
bvzjkezkms.mp4_frame6.jpg
bvzjkezkms.mp4    FAKE
Name: label, dtype: object
bvzjkezkms.mp4_frame7.jpg
bvzjkezkms.mp4    FAKE
Name: label, dtype: object
ekcrtigpab.mp4_frame8.jpg
ekcrtigpab.mp4    REAL
Name: label, dtype: object
acqfdwsrhi.mp4_frame10.jpg
acqfdwsrhi.mp4    FAKE
Name: label, dtype: object
ccfoszqabv.mp4_frame0.jpg
ccfoszqabv.mp4    REAL
Name: label, dtype: object
ccfoszqabv.mp4_frame4.jpg
ccfoszqabv.mp4    REAL
Name: label, dtype: object
bvzjkezkms.mp4_frame3.jpg
bvzjkezkms.mp4    FAKE
Name: label, dtype: object
dlpoieqv

In [113]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,ccfoszqabv.mp4_frame2.jpg,REAL
1,bvzjkezkms.mp4_frame5.jpg,FAKE
2,bvzjkezkms.mp4_frame4.jpg,FAKE
3,ccfoszqabv.mp4_frame3.jpg,REAL
4,ccfoszqabv.mp4_frame1.jpg,REAL


In [114]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loadiimg = image.load_img(train['image'][i], target_size=(224,224,3))ng the image and keeping the target size as (224,224,3)
    
    img = image.load_img(train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 88/88 [00:16<00:00,  5.34it/s]


(88, 224, 224, 3)

In [116]:
# separating the target
y = train['class']


In [117]:
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [118]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [119]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [120]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(70, 7, 7, 512)

In [121]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(18, 7, 7, 512)

In [122]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(70, 7*7*512)
X_test = X_test.reshape(18, 7*7*512)

In [123]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [124]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [125]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [126]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [127]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 0.6735 - accuracy: 0.5714 - val_loss: 0.6988 - val_accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 0s 180ms/step - loss: 0.6984 - accuracy: 0.5143 - val_loss: 0.7608 - val_accuracy: 0.5000
Epoch 3/200
1/1 [==============================] - 0s 200ms/step - loss: 0.8169 - accuracy: 0.5286 - val_loss: 0.7666 - val_accuracy: 0.5000
Epoch 4/200
1/1 [==============================] - 0s 194ms/step - loss: 0.9271 - accuracy: 0.4714 - val_loss: 0.7096 - val_accuracy: 0.5000
Epoch 5/200
1/1 [==============================] - 0s 205ms/step - loss: 0.9170 - accuracy: 0.4429 - val_loss: 0.7064 - val_accuracy: 0.4444
Epoch 6/200
1/1 [==============================] - 0s 201ms/step - loss: 0.9258 - accuracy: 0.5429 - val_loss: 0.7169 - val_accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 190ms/step - loss: 0.9422 - accuracy: 0.5571 - val_loss: 0.7103 - val_accuracy: 0.3889
Epoch 8/200
1/1 

Epoch 59/200
1/1 [==============================] - 0s 282ms/step - loss: 0.7298 - accuracy: 0.5143 - val_loss: 0.7062 - val_accuracy: 0.2778
Epoch 60/200
1/1 [==============================] - 0s 278ms/step - loss: 0.6752 - accuracy: 0.6429 - val_loss: 0.7071 - val_accuracy: 0.3889
Epoch 61/200
1/1 [==============================] - 0s 208ms/step - loss: 0.7258 - accuracy: 0.5000 - val_loss: 0.7075 - val_accuracy: 0.3889
Epoch 62/200
1/1 [==============================] - 0s 273ms/step - loss: 0.7371 - accuracy: 0.4714 - val_loss: 0.7078 - val_accuracy: 0.4444
Epoch 63/200
1/1 [==============================] - 0s 214ms/step - loss: 0.7160 - accuracy: 0.5286 - val_loss: 0.7084 - val_accuracy: 0.4444
Epoch 64/200
1/1 [==============================] - 0s 195ms/step - loss: 0.7078 - accuracy: 0.4714 - val_loss: 0.7086 - val_accuracy: 0.3889
Epoch 65/200
1/1 [==============================] - 0s 203ms/step - loss: 0.7199 - accuracy: 0.4714 - val_loss: 0.7080 - val_accuracy: 0.3889
Epoch 

1/1 [==============================] - 0s 236ms/step - loss: 0.7184 - accuracy: 0.5000 - val_loss: 0.7054 - val_accuracy: 0.2778
Epoch 117/200
1/1 [==============================] - 0s 234ms/step - loss: 0.6947 - accuracy: 0.5143 - val_loss: 0.7053 - val_accuracy: 0.2778
Epoch 118/200
1/1 [==============================] - 0s 237ms/step - loss: 0.7157 - accuracy: 0.5143 - val_loss: 0.7052 - val_accuracy: 0.2778
Epoch 119/200
1/1 [==============================] - 0s 231ms/step - loss: 0.7055 - accuracy: 0.5714 - val_loss: 0.7053 - val_accuracy: 0.2778
Epoch 120/200
1/1 [==============================] - 0s 238ms/step - loss: 0.7236 - accuracy: 0.4571 - val_loss: 0.7052 - val_accuracy: 0.2778
Epoch 121/200
1/1 [==============================] - 0s 241ms/step - loss: 0.7060 - accuracy: 0.5000 - val_loss: 0.7048 - val_accuracy: 0.2778
Epoch 122/200
1/1 [==============================] - 0s 237ms/step - loss: 0.7156 - accuracy: 0.5000 - val_loss: 0.7047 - val_accuracy: 0.2778
Epoch 123/200

1/1 [==============================] - 0s 200ms/step - loss: 0.7179 - accuracy: 0.5286 - val_loss: 0.7144 - val_accuracy: 0.3333
Epoch 174/200
1/1 [==============================] - 0s 203ms/step - loss: 0.6929 - accuracy: 0.5857 - val_loss: 0.7140 - val_accuracy: 0.3333
Epoch 175/200
1/1 [==============================] - 0s 198ms/step - loss: 0.6750 - accuracy: 0.5714 - val_loss: 0.7139 - val_accuracy: 0.3333
Epoch 176/200
1/1 [==============================] - 0s 197ms/step - loss: 0.6721 - accuracy: 0.6000 - val_loss: 0.7137 - val_accuracy: 0.3333
Epoch 177/200
1/1 [==============================] - 0s 205ms/step - loss: 0.7082 - accuracy: 0.4857 - val_loss: 0.7134 - val_accuracy: 0.3333
Epoch 178/200
1/1 [==============================] - 0s 198ms/step - loss: 0.6986 - accuracy: 0.5286 - val_loss: 0.7132 - val_accuracy: 0.3333
Epoch 179/200
1/1 [==============================] - 0s 190ms/step - loss: 0.6942 - accuracy: 0.5429 - val_loss: 0.7123 - val_accuracy: 0.3333
Epoch 180/200